In [ ]:
import sqlite3
import os
from pathlib import Path
from anthropic import Anthropic

key = os.environ["ANTHROPIC_API_KEY"] 
REPO_PATH = Path("/Users/koen/workspace/toolbox")
SCREENPIPE_PATH = REPO_PATH / "data" / "screenpipe" / "db.sqlite" 


In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(SCREENPIPE_PATH)
conn.row_factory = sqlite3.Row

In [ ]:
def get_all_meeting_notes(connection):
    cursor = connection.cursor()
    # Query the audio_transcriptions table
    cursor.execute("SELECT * FROM audio_transcriptions")
    rows = [dict(x) for x in cursor.fetchall()]
    fragments = []

    for row in rows:
        speaker_id = row["speaker_id"]
        transcription = row["transcription"]
        
        fragment = f"Speaker {speaker_id}:   {transcription}"
        fragments.append(fragment)

    full_transcription = "\n".join(fragments)
    return [full_transcription]

In [ ]:
transcriptions = get_all_meeting_notes(conn)

In [ ]:
client = Anthropic(api_key=key)

In [ ]:
def get_todos(transcription):
    prompt = f"""Please analyze this transcription and convert it into a list of actionable todo items. Only respond with the todo items, no other text.
Format each todo item as a bullet point starting with '- '.

Transcription:
{transcription}

Todo items:"""
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
    return response.content[0].text

In [ ]:
todos = get_todos(transcriptions[0])

In [ ]:
print(todos)

NEXT: MANUALLY POST THIS IN CLAUDE AND CHECK IF IT CAN MAKE MULTIPLE TODOs